In [1]:
import sys,os,argparse,time
import numpy as np
import torch
import utils
from datetime import datetime
import pickle
tstart=time.time()

In [2]:
# Arguments
parser=argparse.ArgumentParser(description='xxx')
parser.add_argument('--seed',               default=0,              type=int,     help='(default=%(default)d)')
parser.add_argument('--device',             default='cpu',          type=str,     help='gpu id')
parser.add_argument('--experiment',         default='1_task_groups',       type =str,    help='Mnist or dissertation')
parser.add_argument('--approach',           default='PUGCL',          type =str,    help='Method, always Lifelong Uncertainty-aware learning')
parser.add_argument('--data_path',          default='data/data.csv',     type=str,     help='gpu id')

# Training parameters
parser.add_argument('--output',             default='',             type=str,     help='')
parser.add_argument('--checkpoint_dir',     default='../checkpoints_1_tasks',    type=str,   help='')
parser.add_argument('--n_epochs',           default=100,              type=int,     help='')
parser.add_argument('--batch_size',         default=64,             type=int,     help='')
parser.add_argument('--lr',                 default=0.03,           type=float,   help='')
parser.add_argument('--hidden_size',        default=800,           type=int,     help='')
parser.add_argument('--parameter',          default='',             type=str,     help='')

# UCB HYPER-PARAMETERS
parser.add_argument('--MC_samples',         default='10',           type=int,     help='Number of Monte Carlo samples')
parser.add_argument('--rho',                default='-3',           type=float,   help='Initial rho')
parser.add_argument('--sigma1',             default='0.0',          type=float,   help='STD foor the 1st prior pdf in scaled mixture Gaussian')
parser.add_argument('--sigma2',             default='6.0',          type=float,   help='STD foor the 2nd prior pdf in scaled mixture Gaussian')
parser.add_argument('--pi',                 default='0.25',         type=float,   help='weighting factor for prior')

parser.add_argument('--resume',             default='no',           type=str,     help='resume?')
parser.add_argument('--sti',                default=0,              type=int,     help='starting task?')

parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args=parser.parse_args()
utils.print_arguments(args)

Arguments =
	seed: 0
	device: cpu
	experiment: 1_task_groups
	approach: PUGCL
	data_path: data/data.csv
	output: 
	checkpoint_dir: ../checkpoints_1_tasks
	n_epochs: 100
	batch_size: 64
	lr: 0.03
	hidden_size: 800
	parameter: 
	MC_samples: 10
	rho: -3.0
	sigma1: 0.0
	sigma2: 6.0
	pi: 0.25
	resume: no
	sti: 0
	fff: /Users/jonastjomsland/Library/Jupyter/runtime/kernel-18c4ed29-0a63-47f1-8751-82cf7f1f276c.json


In [3]:
# Set seed for stable results
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# Check if Cuda is available
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("Using device:", args.device)

# Checkpoint
checkpoint = utils.make_directories(args)
args.checkpoint = checkpoint
print()

# PUGCL with two tasks:
from data import dataloader_1_tasks as dataloader

# Import Lifelong Uncertainty-aware Learning approach:
#from bayesian_model.lul import Lul
from training_method import PUGCL

# Import model used:
#from bayesian_model.bayesian_network import BayesianNetwork
from bayesian_model.bayesian_network import BayesianNetwork

Using device: cpu
1_task_groups_PUGCL
Results will be saved in  ../checkpoints_1_tasks/1_task_groups_PUGCL



In [4]:
print()
print("Starting this session on: ")
print(datetime.now().strftime("%Y-%m-%d %H:%M"))

# Load data:
print("Loading data...")
data, task_outputs, input_size = dataloader.get(data_path=args.data_path)
print("Input size =", input_size, "\nTask info =", task_outputs)
print("Number of data samples: ", len(data[0]['train']['x']))
args.num_tasks = len(task_outputs)
args.input_size = input_size
args.task_outputs = task_outputs
pickle.dump(data, open( "data/data_1_task.p", "wb" ))

# Initialize Bayesian network
print("Initializing network...")
model = BayesianNetwork(args).to(args.device)

# Initialize Lul approach
print("Initialize Lifelong Uncertainty-aware Learning")
approach = PUGCL(model, args=args)
print("-"*100)

# Check wether resuming:
if args.resume == "yes":
    checkpoint = torch.load(os.path.join(args.checkpoint, 'model_{}.pth.tar'.format(args.sti)))
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device=args.device)
else:
    args.sti = 0


Starting this session on: 
2020-05-15 16:58
Loading data...
Input size = [1, 29] 
Task info = [(0, 16)]
Number of data samples:  10000
Initializing network...
Initialize Lifelong Uncertainty-aware Learning
----------------------------------------------------------------------------------------------------


/Users/jonastjomsland/anaconda3/envs/ucb/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [5]:
# Iterate over the tasks:
loss = np.zeros((len(task_outputs), len(task_outputs)), dtype=np.float32)
for task, n_class in task_outputs[args.sti:]:
    print('*'*100)
    print('Task {:2d} ({:s})'.format(task, data[task]['name']))
    print('*'*100)

    # Get data:
    xtrain = data[task]['train']['x'][:,1:].type(torch.float32).to(args.device)
    ytrain = data[task]['train']['y'].type(torch.float32).to(args.device)
    xvalid = data[task]['valid']['x'][:,1:].type(torch.float32).to(args.device)
    yvalid = data[task]['valid']['y'].type(torch.float32).to(args.device)

    # Start training
    print("Starting training for the tasks in group: ", task)
    approach.train(task, xtrain, ytrain, xvalid, yvalid)
    print('_'*100)

    # Validate for this task group:
    for u in range(task+1):
        xtest = data[u]['test']['x'][:,1:].type(torch.float32).to(args.device)
        ytest = data[u]['test']['y'].type(torch.float32).to(args.device)
        test_loss = approach.eval(u, xtest, ytest, debug=True)
        print("Test on task {:2d} - {:15s}: loss={:.3f}".format(u, data[u]['name'], test_loss))
        loss[task, u] = test_loss

    # Save
    print("Saving at " + args.checkpoint)
    np.savetxt(os.path.join(args.checkpoint, '{}_{}_{}.txt'.format(args.experiment, args.approach, args.seed)), loss, '%.5f')

****************************************************************************************************
Task  0 (All actions)
****************************************************************************************************
Starting training for the tasks in group:  0
Batch: 0/10000 

../torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)


Batch: 9984/10000 | Epoch   1, time=627.6ms/  9.9ms | Training loss: 1.586 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   2, time=608.2ms/  9.4ms | Training loss: 1.492 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   3, time=598.6ms/  9.3ms | Training loss: 1.458 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   4, time=596.9ms/  9.1ms | Training loss: 1.431 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   5, time=596.3ms/  9.2ms | Training loss: 1.411 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   6, time=594.3ms/  8.8ms | Training loss: 1.398 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   7, time=592.8ms/  9.3ms | Training loss: 1.380 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch   8, time=588.0ms/  9.1ms | Training loss: 1.386 | Learning rate: 0.030 |
Batch: 9984/10000 | Epoch   9, time=585.8ms/  9.1ms | Training loss: 1.374 | Learning rate: 0.030 | *
Batch: 9984/10000 | Epoch  10, time=587.0ms/  8.8ms | Training loss: 1.367 | Learnin